**KHELLADI Sid Ali**

**DOUID Mohamed**


# I) Back-end

In [166]:
# 1
# 1.1
import xml.etree.ElementTree as ET
# 1.2
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# 2
from collections import Counter, defaultdict
from nltk import pos_tag
import pandas as pd
import numpy as np
# 4
from gensim import corpora, models
# 5
from transformers import MarianMTModel, MarianTokenizer



In [52]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**1) Prétraitement des Données**

**1.1) Extraction des \<AbstractText\>**

In [54]:
def extract_abstract_texts(file_path, max_abstracts=25):
    tree = ET.parse(file_path)
    root = tree.getroot()
    abstracts = []
    
    for i, abstract in enumerate(root.iter('AbstractText')):
        if i < max_abstracts:
            abstracts.append(abstract.text)
        else:
            break
    
    return abstracts

**1.2) Segmentation en phrases**

In [55]:
def segment_sentences(abstracts):
    sentences = [sent_tokenize(abstract) for abstract in abstracts if isinstance(abstract, str)]
    return [sentence for sublist in sentences for sentence in sublist]


**1.3) Creation du fichier global**

In [56]:
def create_global_file(sentences, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(sentence + '\n')


In [75]:
# 1.1
abstracts = extract_abstract_texts("sample-0001.xml")


In [58]:
# 1.2
sentences = segment_sentences(abstracts)

In [59]:
# 1.3
output_path = "output.txt"

create_global_file(sentences, output_path)


**2) Etiquetage Morphosyntaxique (POS Tagging)**

In [60]:
def compute_transition_matrices(tags):
    unigram_counts = Counter(tags)
    bigram_counts = Counter(zip(tags, tags[1:]))
    
    total_unigrams = sum(unigram_counts.values())
    total_bigrams = sum(bigram_counts.values())

    unigram_probs = {tag: count / total_unigrams for tag, count in unigram_counts.items()}
    bigram_probs = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}

    return unigram_probs, bigram_probs

In [65]:
words = [word for sentence in sentences for word in word_tokenize(sentence)]
tags = [tag for word, tag in pos_tag(words)]

In [199]:
# Calcul des matrices de transition
unigram_probs, bigram_probs = compute_transition_matrices(tags)
list_tags = []
for key,value in unigram_probs.items():
    print(key,":",value)
    if key not in list_tags:
        list_tags.append(key)
print(list_tags)

DT : 0.07128446536650974
JJ : 0.10793544048419637
NN : 0.18897108271687962
NNP : 0.05514458641560188
IN : 0.1207128446536651
NNS : 0.06657700067249496
VBD : 0.03026227303295225
( : 0.01714862138533961
) : 0.01714862138533961
CD : 0.05211835911230666
TO : 0.019166106254203095
VB : 0.016812373907195696
CC : 0.03194351042367182
, : 0.04404841963685272
. : 0.03732347007397444
VBN : 0.026899798251513115
NNPS : 0.0006724949562878278
VBP : 0.008742434431741762
RB : 0.02051109616677875
VBG : 0.011432414256893073
JJR : 0.006724949562878279
VBZ : 0.015131136516476126
`` : 0.0003362474781439139
'' : 0.0003362474781439139
PRP : 0.009078681909885675
WP : 0.0020174848688634837
RBR : 0.0003362474781439139
$ : 0.0006724949562878278
MD : 0.005043712172158709
WDT : 0.003698722259583053
WRB : 0.0010087424344317419
EX : 0.0013449899125756557
: : 0.004034969737726967
PRP$ : 0.003026227303295225
JJS : 0.0006724949562878278
POS : 0.0010087424344317419
RBS : 0.0003362474781439139
PDT : 0.0003362474781439139
[

**2.1) Dispalying Unigram probabilities**

In [205]:

df = pd.DataFrame(unigram_probs, columns =list_tags,index=["values"])
print(df)

              DT        JJ        NN       NNP        IN       NNS       VBD  \
values  0.071284  0.107935  0.188971  0.055145  0.120713  0.066577  0.030262   

               (         )        CD  ...        MD       WDT       WRB  \
values  0.017149  0.017149  0.052118  ...  0.005044  0.003699  0.001009   

              EX         :      PRP$       JJS       POS       RBS       PDT  
values  0.001345  0.004035  0.003026  0.000672  0.001009  0.000336  0.000336  

[1 rows x 38 columns]


**2.1.2) Dispalying Unigram probabilities on Excel file**

In [120]:
excel_file_unigram_probs = 'unigram_probs.xlsx'
df.to_excel(excel_file_unigram_probs, index=False)

print(f"Le DataFrame a été enregistré dans le fichier {nom_fichier_excel}")


Le DataFrame a été enregistré dans le fichier unigram_probs.xlsx


**2.2) Dispalying Bigram probabilities**

In [203]:
premieres_parties = [cle[0] for cle in bigram_probs.keys()]

deuxieme_parties = [cle[1] for cle in bigram_probs.keys()]

val_bigram_probs_of_tag = [cle for cle in bigram_probs.values()]

val_bigram_probs_of_tag

for i in range(38):
    if listOfTags[i] == list_tags[i]:
        print(i,"listOfTags:",listOfTags[i],"list_tags:",list_tags[i])
    

0 listOfTags: DT list_tags: DT
1 listOfTags: JJ list_tags: JJ
2 listOfTags: NN list_tags: NN
3 listOfTags: NNP list_tags: NNP
4 listOfTags: IN list_tags: IN
5 listOfTags: NNS list_tags: NNS
6 listOfTags: VBD list_tags: VBD
7 listOfTags: ( list_tags: (
8 listOfTags: ) list_tags: )
9 listOfTags: CD list_tags: CD
10 listOfTags: TO list_tags: TO
11 listOfTags: VB list_tags: VB
12 listOfTags: CC list_tags: CC
13 listOfTags: , list_tags: ,
14 listOfTags: . list_tags: .
15 listOfTags: VBN list_tags: VBN
16 listOfTags: NNPS list_tags: NNPS
17 listOfTags: VBP list_tags: VBP
18 listOfTags: RB list_tags: RB
19 listOfTags: VBG list_tags: VBG
20 listOfTags: JJR list_tags: JJR
21 listOfTags: VBZ list_tags: VBZ
22 listOfTags: `` list_tags: ``
23 listOfTags: '' list_tags: ''
24 listOfTags: PRP list_tags: PRP
25 listOfTags: WP list_tags: WP
26 listOfTags: RBR list_tags: RBR
27 listOfTags: $ list_tags: $
28 listOfTags: MD list_tags: MD
29 listOfTags: WDT list_tags: WDT
30 listOfTags: WRB list_tags: WRB


In [170]:
bigram_probs_mat = np.zeros((len(list_tags),len(list_tags)))
bigram_probs_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [178]:
def getIndexOfTag(tag,list_tags):
    for i in range(len(list_tags)):
        if list_tags[i] == tag:
            return i

In [206]:
for i in range(len(premieres_parties)):
    bigram_probs_mat[
        getIndexOfTag(
            premieres_parties[i],
            list_tags
        ),
        getIndexOfTag(
            deuxieme_parties[i],
            list_tags
        )
    ] = val_bigram_probs_of_tag[i]
    
bigram_probs_mat

df_Bigram_probs = pd.DataFrame(bigram_probs_mat, columns=listOfTags, index=listOfTags)
df_Bigram_probs

,DT,JJ,NN,NNP,IN,NNS,VBD,(,),CD,...,MD,WDT,WRB,EX,:,PRP$,JJS,POS,RBS,PDT
DT,0.000000,0.018500,0.029936,0.007064,0.001682,0.006727,0.000673,0.000000,0.000000,0.002018,...,0.000336,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000336,0.000000
JJ,0.000000,0.012109,0.057181,0.004036,0.003700,0.018163,0.000000,0.000000,0.001009,0.002018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NN,0.000336,0.005382,0.024891,0.005718,0.052136,0.010764,0.008073,0.008073,0.006054,0.001682,...,0.001345,0.001682,0.000000,0.000000,0.002018,0.000000,0.000000,0.000673,0.000000,0.000000
NNP,0.000000,0.002018,0.006727,0.009418,0.004373,0.003364,0.000673,0.004709,0.005718,0.008409,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000336,0.000000,0.000000
IN,0.029263,0.027918,0.025563,0.006727,0.002691,0.006054,0.000336,0.000000,0.000000,0.010427,...,0.000000,0.000673,0.000000,0.000000,0.000000,0.001009,0.000673,0.000000,0.000000,0.000000
NNS,0.000000,0.001009,0.000000,0.000000,0.020182,0.000000,0.006727,0.002018,0.001345,0.000336,...,0.001009,0.000673,0.000000,0.000000,0.001345,0.000000,0.000000,0.000000,0.000000,0.000000
VBD,0.004709,0.004036,0.001345,0.000336,0.002355,0.001009,0.000000,0.000336,0.000000,0.001682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000336
(,0.000336,0.002691,0.000336,0.009418,0.000336,0.000000,0.000000,0.000000,0.000000,0.003364,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
),0.000336,0.000000,0.001009,0.000000,0.002691,0.000000,0.001682,0.000000,0.000000,0.000000,...,0.001345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CD,0.000000,0.004036,0.021191,0.000000,0.002691,0.011436,0.000336,0.001009,0.003027,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000673,0.000000,0.000000,0.000000,0.000000,0.000000


In [208]:
excel_file_bigram_probs = 'bigram_probs.xlsx'
df_Bigram_probs.to_excel(excel_file_bigram_probs, index=True)

print(f"Le DataFrame a été enregistré dans le fichier {nom_fichier_excel}")


Le DataFrame a été enregistré dans le fichier unigram_probs.xlsx


# !!!! Begin test zone !!!!

# !!!! End test zone !!!!

**3) Reconnaissance d'Entités Nommées (NER)**

In [42]:
def compute_ner_transition_matrices(tags):
    unigram_counts = Counter(tags)
    bigram_counts = Counter(zip(tags, tags[1:]))

    total_unigrams = sum(unigram_counts.values())
    total_bigrams = sum(bigram_counts.values())

    unigram_probs = {tag: count / total_unigrams for tag, count in unigram_counts.items()}
    bigram_probs = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}

    return unigram_probs, bigram_probs


**4) Modélisation des Sujets (Topics Modeling)**

In [45]:
def topic_modeling(texts, num_topics=3):
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    ldamodel = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_words=4)
    return topics


**5) Traduction**

In [48]:
def translate_text(text, src_lang='en', tgt_lang='fr'):
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    
    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    translation = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translation


# II) Front-end 

In [49]:
from flask import Flask, request, jsonify

**1) Interface Utilisateur**

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello, World!'

@app.route('/api/data', methods=['POST'])
def get_data():
    data = request.get_json()
    response = {'received': data}
    return jsonify(response)

if __name__ == '__main__':
    try:
        app.run(debug=True, use_reloader=False)
    except Exception as e:
        print(f"An error occurred: {e}")
